In [1]:
from datetime import datetime
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd

# Set tracking URI and experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")
client = MlflowClient()

# Define model names for comparison
model_names = [
    "milk-price-predictor-xgb",
    "milk-price-predictor-rf"
]

# Fetch latest versions and their RMSEs
model_info = []
for name in model_names:
    versions = client.get_latest_versions(name, stages=["None"])
    for v in versions:
        run = client.get_run(v.run_id)
        rmse = run.data.metrics.get("final_rmse")
        model_info.append({
            "model_name": name,
            "version": v.version,
            "run_id": v.run_id,
            "rmse": rmse
        })

# Convert to DataFrame and determine best model
df_models = pd.DataFrame(model_info)
best_model = df_models.sort_values("rmse").iloc[0]

# Promote best model to Staging
client.transition_model_version_stage(
    name=best_model.model_name,
    version=best_model.version,
    stage="Staging",
    archive_existing_versions=True
)

best_model_info = {
    "model_name": best_model.model_name,
    "version": best_model.version,
    "rmse": best_model.rmse,
    "promoted_stage": "Staging",
    "promotion_time": datetime.now().isoformat()
}

best_model_info


/tmp/ipykernel_26783/77213842.py:19: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name, stages=["None"])
/tmp/ipykernel_26783/77213842.py:35: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


{'model_name': 'milk-price-predictor-xgb',
 'version': '1',
 'rmse': np.float64(0.2554389777452065),
 'promoted_stage': 'Staging',
 'promotion_time': '2025-07-31T12:36:31.779459'}